## GetEdgeFeature

In [1]:
import torch
import numpy as np

def knn(data, k=20)->torch.Tensor:
    """Construct edge feature for each point
    Args:
      point_cloud: (batch_size, num_points, num_dims)
      k: int number of neighbours

    Returns:
      idx: shape:(batch_size, num_points, nums_neighours,)
    """
    dists_matrix = torch.cdist(data, data)
    #print(dists_matrix.shape)
    _, idx = dists_matrix.topk(k+1, dim=-1, largest=False)  # +1 the point itself is included
    return idx[...,1:] # not include the point itself

def get_edge_feature(point_cloud, idx=None, k=20,device="cpu"):
    """Construct edge feature for each point
    Args:
      point_cloud: (batch_size, num_points, num_dims)
      idx: (batch_size, num_points, neighbours)
      k: int
      device: cpu/cuda

    Returns:
      features: (batch_size, num_dims ,num_points, k)
    """
    point_cloud = point_cloud.to(device)
    batch_size = point_cloud.shape[0]
    num_points = point_cloud.shape[1]

    if(idx==None):
        idx = knn(point_cloud,k=k) # (batch_size, num_points, nums_neighours,)

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1) * num_points # create the base index for mapping
    idx = idx.to(device=device)
    idx = idx + idx_base #[0...0...0]->[0...100...200]
    idx=idx.view(-1) # flatten it -> tensor([  0,  56,  25,  ..., 225, 222, 271], device='cuda:0') e.g: [K01,K02,K03,K11,K12,K13...] shape = (B*N*K) 
   
    num_dims = point_cloud.shape[2]

    # feature : turn neighbour index in idx to coordinate
    feature = point_cloud.view(batch_size*num_points, -1)[idx, :] # feature : B*N*F -> BN * F -> (B*N*K) * F
    # feature : reshape into (Batch_size * Num_points *Nums_neigbours * Features)
    feature = feature.view(batch_size, num_points, k, num_dims)
    
    # pointcloud : create replicate of the self point up to k for matching feature - size B*N*K(repeated)*F 
    point_cloud = point_cloud.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1) 

    # feature size B*N*K*F -> B*N*K*2F (feature-x || x)
    feature = torch.cat((feature-point_cloud, point_cloud), dim=3)

    # (B * 2F * N * K) for later conv each coordinate(F)
    feature=feature.permute(0,3,1,2).contiguous()

    return feature

## EdgeConv

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EdgeConv(nn.Module):
    def __init__(self, in_channels, out_channels, num_neighbours=20,device="cpu"):
        """Setup EdgeConv
        Args:
        in_channels: int
        out_channels: int
        num_neighbours: int
        """
        super(EdgeConv, self).__init__()
        self.device=device
        self.k= num_neighbours
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels*2, out_channels=out_channels, kernel_size=1, bias=False,device=self.device),
            nn.BatchNorm2d(out_channels,device=self.device),
            nn.LeakyReLU(negative_slope=0.2)
        )

    def forward(self,x):
        """Setup EdgeConv
        Args:
        x: shape - (batch_size, num_points, num_dims)

        Returns:
        features: (batch_size, num_dims, num_points, num_neigbours)
        """
        x = get_edge_feature(x, k=self.k,device=self.device) #(batch_size, num_points, dim) -> (batch_size, dim*2, num_points ,k)
        x = self.conv(x)
        # for each point pick the largest k (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)
        x = x.max(dim=-1, keepdim=False)[0]
        x = x.permute(0,2,1).contiguous()
        return x

## DGCNN - Classification - Teacher

In [3]:
class DGCNN(nn.Module):
    def __init__(self, num_neighbours=20,out_channels=40,dropout_rate =0.3,device="cpu"):
        super(DGCNN,self).__init__()
        self.inChannels=[3,64,64,128,256]
        self.edgeConv0 = EdgeConv(in_channels=3,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv1 = EdgeConv(in_channels=64,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv2 = EdgeConv(in_channels=64,out_channels=128,num_neighbours=num_neighbours,device=device)
        self.edgeConv3 = EdgeConv(in_channels=128,out_channels=256,num_neighbours=num_neighbours,device=device)

        self.edgeConv4 = EdgeConv(in_channels=512,out_channels=1024,num_neighbours=num_neighbours,device=device)

        self.linear1 = nn.Linear(2048, 512, bias=False,device=device)
        self.bn1 = nn.BatchNorm1d(512,device=device)
        self.drop1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(512, 256, bias=False,device=device)
        self.bn2 = nn.BatchNorm1d(256,device=device)
        self.drop2 = nn.Dropout(dropout_rate)
        self.linear3 = nn.Linear(256,out_channels, bias=False,device=device)


    def forward(self,x):
        x0=self.edgeConv0(x)
        #print("x0:",x0.shape)
        x1=self.edgeConv1(x0)
        x2=self.edgeConv2(x1)
        x3=self.edgeConv3(x2)

        x=torch.cat((x0,x1,x2,x3),dim=2)
        
        x= self.edgeConv4(x) # (batch_size, num_points ,64+64+128+256) -> (batch_size, num_points, emb_dims(1024))
        
        #todo 
        # maxpool and avgpool
        x= x.permute(0,2,1).contiguous()
        maxPoolX = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        avgPoolX = F.adaptive_avg_pool1d(x,1).view(x.shape[0],-1)
        x=torch.cat((maxPoolX,avgPoolX),1) #(batch_size, 2048)
        
        #mlp[512,256,c(40)]
        x= F.leaky_relu(self.bn1(self.linear1(x)))
        x=self.drop1(x)
        x= F.leaky_relu(self.bn2(self.linear2(x)))
        x=self.drop2(x)
        x= self.linear3(x)
        # output

        return x

## DGCNN - Classification - Student

In [4]:
class DGCNN_S(nn.Module):
    def __init__(self, num_neighbours=10,out_channels=40,device="cpu"):
        super(DGCNN_S,self).__init__()
        self.inChannels=[3,64,64,128,256]
        self.edgeConv0 = EdgeConv(in_channels=3,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv1 = EdgeConv(in_channels=64,out_channels=64,num_neighbours=num_neighbours,device=device)
        self.edgeConv2 = EdgeConv(in_channels=64,out_channels=128,num_neighbours=num_neighbours,device=device)

        self.edgeConv4 = EdgeConv(in_channels=256,out_channels=512,num_neighbours=num_neighbours,device=device)

        self.linear1 = nn.Linear(512, 256, bias=False,device=device)
        self.bn1 = nn.BatchNorm1d(256,device=device)
        self.linear2 = nn.Linear(256,out_channels, bias=False,device=device)


    def forward(self,x):
        x0=self.edgeConv0(x)
        #print("x0:",x0.shape)
        x1=self.edgeConv1(x0)
        x2=self.edgeConv2(x1)

        x=torch.cat((x0,x1,x2),dim=2)
        
        x= self.edgeConv4(x) # (batch_size, num_points ,64+64+128) -> (batch_size, num_points, emb_dims(256))
        
        #todo 
        # maxpool and avgpool
        x= x.permute(0,2,1).contiguous()
        x = F.adaptive_max_pool1d(x,1).view(x.shape[0],-1)
        
        #mlp[512,256,c(40)]
        x= F.leaky_relu(self.bn1(self.linear1(x)))
        x= self.linear2(x)
        # output

        return x

## DataSet

In [5]:
import os
import glob
import h5py
import numpy as np
from torch.utils.data import Dataset

data_path = "/kaggle/input/model4/modelnet40_ply_hdf5_2048/modelnet40_ply_hdf5_2048"

def load_data_cls(partition):
    all_data = []
    all_label = []
    for h5_name in glob.glob(os.path.join(data_path, '*%s*.h5'%partition)):
        f = h5py.File(h5_name, 'r')
        data = f['data'][:].astype('float32')
        label = f['label'][:].astype('int64')
        f.close()
        all_data.append(data)
        all_label.append(label)
    all_data = np.concatenate(all_data, axis=0)
    all_label = np.concatenate(all_label, axis=0)
    return all_data, all_label

def translate_pointcloud(pointcloud):
    xyz1 = np.random.uniform(low=2./3., high=3./2., size=[3])
    xyz2 = np.random.uniform(low=-0.2, high=0.2, size=[3])
       
    translated_pointcloud = np.add(np.multiply(pointcloud, xyz1), xyz2).astype('float32')
    return translated_pointcloud

class ModelNet40(Dataset):
    def __init__(self, num_points, partition='train'):
        self.data, self.label = load_data_cls(partition)
        self.num_points = num_points
        self.partition = partition        

    def __getitem__(self, item):
        pointcloud = self.data[item][:self.num_points]
        label = self.label[item]
        if self.partition == 'train':
            pointcloud = translate_pointcloud(pointcloud)
            np.random.shuffle(pointcloud)
        return pointcloud, label

    def __len__(self):
        return self.data.shape[0]

## Teacher Training

In [ ]:
traning
from torch.utils.data import DataLoader
import sklearn.metrics as metrics
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

numPoints = 1024
batchSize = 16
testBatchSize = 8
_lr = 0.001
epochs = 50
PATH = "./teacher_model.t7"

train_loader = DataLoader(ModelNet40(partition='train',num_points=numPoints), num_workers=2,
                              batch_size=batchSize, shuffle=True, drop_last=True)
test_loader = DataLoader(ModelNet40(partition='test', num_points=numPoints), num_workers=2,
                             batch_size=testBatchSize, shuffle=True, drop_last=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dgcnn = DGCNN(device=device)

opt = optim.Adam(dgcnn.parameters(), lr=_lr, weight_decay=1e-4)

scheduler = CosineAnnealingLR(opt, epochs, eta_min=_lr)

best_test_acc = 0

for epoch in range(epochs):
    ####################
    # Train
    ####################
    train_loss = 0.0
    count = 0.0
    dgcnn.train()
    train_pred = []
    train_true = []
    for data, label in train_loader:
        data, label = data.to(device), label.to(device).squeeze()
        batch_size = data.size()[0]
        opt.zero_grad()
        logits = dgcnn(data)
        loss = F.cross_entropy(logits, label, reduction='mean')
        loss.backward()
        opt.step()
        scheduler.step()
        preds = logits.max(dim=1)[1]
        count += batch_size
        train_loss += loss.item() * batch_size
        train_true.append(label.cpu().numpy())
        train_pred.append(preds.detach().cpu().numpy())
    train_true = np.concatenate(train_true)
    train_pred = np.concatenate(train_pred)
    outstr = 'Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f' % (epoch,train_loss*1.0/count,metrics.accuracy_score
                                                (train_true, train_pred),metrics.balanced_accuracy_score
                                                (train_true, train_pred))
    print(outstr)
    
    ####################
    # Test
    ####################
    test_loss = 0.0
    count = 0.0
    dgcnn.eval()
    test_pred = []
    test_true = []
    for data, label in test_loader:
        data, label = data.to(device), label.to(device).squeeze()
        batch_size = data.size()[0]
        logits = dgcnn(data)
        loss = F.cross_entropy(logits, label, reduction='mean')
        preds = logits.max(dim=1)[1]
        count += batch_size
        test_loss += loss.item() * batch_size
        test_true.append(label.cpu().numpy())
        test_pred.append(preds.detach().cpu().numpy())
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)
    test_acc = metrics.accuracy_score(test_true, test_pred)
    avg_per_class_acc = metrics.balanced_accuracy_score(test_true, test_pred)
    outstr = 'Test %d, loss: %.6f, test acc: %.6f, test avg acc: %.6f' % (epoch,test_loss*1.0/count,test_acc,avg_per_class_acc)
    print(outstr)
    if test_acc >= best_test_acc:
        best_test_acc = test_acc
        torch.save(dgcnn.state_dict(), PATH)

## Knowledge Distillation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

lr = 0.001
num_epochs = 50
num_points = 1024
batch_size = 16
temperature = 2.0
apha = 0.5
beta = 0.5
PATH = "/kaggle/input/model5/teacher_model.t7"

class KnowledgeDistillationLoss(nn.Module):
    def __init__(self, temperature=1.0):
        super(KnowledgeDistillationLoss, self).__init__()
        self.temperature = temperature

    def forward(self, student_logits, teacher_logits):
        student_probs = F.softmax(student_logits / self.temperature, dim=1)
        teacher_probs = F.softmax(teacher_logits / self.temperature, dim=1)
        loss = F.kl_div(student_probs.log(), teacher_probs, reduction='batchmean')
        return loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader = DataLoader(ModelNet40(num_points, 'train'), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(ModelNet40(num_points, 'test'), batch_size=batch_size, shuffle=False)
teacher_model = DGCNN(device=device)
teacher_model.load_state_dict(torch.load(PATH,map_location=device))
student_model = DGCNN_S(device=device)
optimizer = optim.Adam(student_model.parameters(), lr)
distillation_loss = KnowledgeDistillationLoss(temperature=temperature)

best_test_acc = 0
for epoch in range(num_epochs):
    #train
    student_model.train()
    total_loss = 0
    for data, label in train_loader:
        data, label = data.to(device), label.to(device).squeeze()
        optimizer.zero_grad()
        student_logits = student_model(data)
        teacher_logits = teacher_model(data)
        loss = apha * distillation_loss(student_logits, teacher_logits) + beta * F.cross_entropy(student_logits, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {total_loss / len(train_loader):.4f}')
    #test
    student_model.eval()
    total = 0
    correct = 0
    total_loss = 0
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.to(device), label.to(device).squeeze()
            output = student_model(data)
            loss = F.cross_entropy(output, label)
            total_loss += loss.item()
            _, predicted = output.max(1)
            total += label.size(0)
            correct += predicted.eq(label).sum().item()
    test_acc = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Test Loss: {total_loss / len(test_loader):.4f}, Test Acc: {test_acc:.4f}')
    if test_acc >= best_test_acc:
        best_test_acc = test_acc
        torch.save(student_model.state_dict(), '/kaggle/working/student_model_t2.t7')


## Evaluation

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import sklearn.metrics as metrics

num_points = 1024
batch_size = 8

TPATH = r"/kaggle/input/model5/teacher_model.t7"
SPATH = r"/kaggle/input/model5/student_model_t2.t7"

test_loader = DataLoader(ModelNet40(num_points, 'test'), batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = DGCNN(device=device)
student_model = DGCNN_S(device=device)

teacher_model.load_state_dict(torch.load(TPATH,map_location=device))
student_model.load_state_dict(torch.load(SPATH,map_location=device))

teacher_model.eval()
student_model.eval()

count = 0.0
test_true = []

t_test_pred = []
s_test_pred = []

for data, label in test_loader:
    data, label = data.to(device), label.to(device).squeeze()
    soutput = student_model(data)
    toutput = teacher_model(data)
    _, spreds = soutput.max(1)
    _, tpreds = toutput.max(1)
    count += label.size(0)
    test_true.append(label.cpu().numpy())
    t_test_pred.append(tpreds.detach().cpu().numpy())
    s_test_pred.append(spreds.detach().cpu().numpy())
test_true = np.concatenate(test_true)
s_test_pred = np.concatenate(s_test_pred)
t_test_pred = np.concatenate(t_test_pred)
s_test_acc = metrics.accuracy_score(test_true, s_test_pred)
t_test_acc = metrics.accuracy_score(test_true, t_test_pred)
t_avg_per_class_acc = metrics.balanced_accuracy_score(test_true, t_test_pred)
s_avg_per_class_acc = metrics.balanced_accuracy_score(test_true, s_test_pred)
t_outstr = 'teacher: test acc: %.6f, test avg acc: %.6f' % (t_test_acc,t_avg_per_class_acc)
s_outstr = 'student: test acc: %.6f, test avg acc: %.6f' % (s_test_acc,s_avg_per_class_acc)
print(t_outstr)
print(s_outstr)

teacher: test acc: 0.896677, test avg acc: 0.858017
student: test acc: 0.920989, test avg acc: 0.879750
